In [23]:
# vertix class
import sys

class Vertix(object):
    def __init__(self, node):
        self.id = node
        self.adjacent = {}
        self.distance = sys.maxsize # why ?
        self.visited = False
        self.prev = None # why ?
        self.color = 'white'

    def addNeighbor(self, neighbor, weight = 0):
        self.adjacent[neighbor] = weight # for weighted graph

    def getConnections(self):
        return self.adjacent.keys()
    
    def getVertixId(self):
        return self.id
    
    def getWeight(self, neighbor):
        return self.adjacent[neighbor]
    
    def setColor(self, color):
        self.color = color

    def setVisited(self):
        self.visited = True

    def getVisited(self):
        return self.visited
    
    def setDistance(self, distance):
        self.distance = distance

    def getDistance(self):
        return self.distance
    
    def setPrev(self, prev):
        self.prev = prev

    def getPrev(self):
        return self.prev
    
    # def __str__(self):
        # return str(self.id) + 'adjacent: ' + str([neighbor.id for neighbor in self.adjacent])



In [2]:
class Graph(object):
    def __init__(self):
        self.vertDict = dict()
        self.numOfVertix = 0

    def __iter__(self): # return iterable object
        return iter(self.vertDict.values()) # why ?
    
    def addVertix(self, node):
        self.numOfVertix += 1
        newNode = Vertix(node)
        self.vertDict[node] = newNode
        return newNode

    def getVertix(self, node):
        if node in self.vertDict:
            return self.vertDict[node]
        else:
            return None
    def addEdge(self, frm, to, cost= 0):
        if frm not in self.vertDict:
            self.addVertix(frm)

        if to not in self.vertDict:
            self.addVertix(to)

        # for directed graph
        self.vertDict[frm].addNeighbor(to, cost)

    def getVertices(self):
        return self.vertDict.keys()
    
    def getEdges(self):
        edges = []
        for v in self.__iter__():
            for neigh in v.getConnections():
                v_id = v.getVertixId()
                neigh_id = self.vertDict[neigh].getVertixId()
                edges.append([v_id, neigh_id, v.getWeight(neigh)])

        return edges


In [3]:

graph = Graph()
graph.addVertix('a')
graph.addVertix('b')
graph.addVertix('c')
graph.addVertix('d')

graph.addEdge('a', 'b', 4)
graph.addEdge('a', 'c', 1)
graph.addEdge('c', 'b', 2)
graph.addEdge('c', 'd', 4)

# print(graph.getEdges())

graph.vertDict



{'a': <__main__.Vertix at 0x1eaba6927d0>,
 'b': <__main__.Vertix at 0x1eaba692490>,
 'c': <__main__.Vertix at 0x1eaba692510>,
 'd': <__main__.Vertix at 0x1eaba692d10>}

Depth First Search

In [4]:
# recursive approach

def dfsTraversal(graph):
    visited = dict()
    for vertix in graph: # returns Class Vertix
        if vertix not in visited:
            dfs(graph, vertix, visited)

def dfs(graph, vertix, visited):
    visited[vertix] = True
    print(vertix.getVertixId())
    for neigh in vertix.getConnections():
        neigh = graph.vertDict[neigh]
        # print("flag 1: ", neigh)
        if neigh not in visited:
            dfs(graph, neigh, visited)


In [5]:
for i in graph:
    print(i)

In [6]:
# example
dfsTraversal(graph)

a
b
c
d


In [7]:
import queue

def BFSTraversal(graph, start):
    # start = graph.getVertix(start)
    start.prev = None
    start.distance = 0
    vertQueue = queue.Queue()
    vertQueue.put(start)

    while (vertQueue.qsize() > 0):
        currVertix = vertQueue.get()
        print(currVertix.getVertixId())
        for nbr in currVertix.getConnections():
            nbr = graph.vertDict[nbr] # nbr is just a string here
            if nbr.color == 'white':
                nbr.setColor('gray')
                nbr.distance = currVertix.distance + 1
                vertQueue.put(nbr)
        currVertix.setColor('black')

def BFS(graph):
    for vertix in graph: # vertix is class Vertix here
        if vertix.color == 'white':
            BFSTraversal(graph, vertix)

In [8]:
BFS(graph)

a
b
c
d


In [ ]:
# shortest-path algorithm (unweighted edge)
# we need to set unvisited distance as -1

def unweightedShortedPath(graph, s):
    """ Computes the shortest distance from source node s to each vertix in the graph
    Argument:
        graph          -- object of class Graph
        s              -- id of source vertix
    Return:
        graph.vertDict -- dictionary of the vertices of the graph
    """
    source = graph.getVertix(s)
    source.distance = 0
    source.prev = None
    vertQueue = queue.Queue()
    vertQueue.put(source)
 
    while(vertQueue.qsize() > 0):
        curr = vertQueue.get()
        for nbr in curr.getConnections():
            nbr = graph.vertDict[nbr]
            
            if nbr.distance == -1:
                nbr.prev = curr
                nbr.distance = curr.distance + 1
                vertQueue.put(nbr)

    return graph.vertDict.values()
            

In [18]:
for key in unweightedShortedPath(graph, 'a'):
    print(key.getVertixId(), '--', key.distance)

a -- 0
b -- 1
c -- 1
d -- 2


In [70]:
# shortest-path for weighted edged graph
# dijkstra (initial distance should be large first (sys.maxsize))

import heapq

def dijkstra(graph, s):
    """ Computes the shortest distance between source starting vertix to each vertix
    Argument:
        graph          -- an instance of Graph class
        s              -- id of a strating vertix
    Returns:
        graph.vertDict -- dictionary of all vertices of the graph
    """
    start = graph.getVertix(s)
    start.setDistance(0)
    unvisited = [(v.getDistance(), v) for v in graph.vertDict.values()]
    print(unvisited)
    heapq.heapify(unvisited)

    while len(unvisited):
        curr_tuple = heapq.heappop(unvisited) # returns tuple
        curr = curr_tuple[1] # the vertix is the second element of the tuple
        curr.setVisited()

        for next_ in curr.adjacent:
            next = graph.getVertix(next_)
            
            if next.getVisited():
                continue

            newDistance = curr.getDistance() + curr.getWeight(next_)
            
            if newDistance < next.getDistance():
                next.setDistance(newDistance)
                next.setPrev(curr)

        # empty unvisited
        while len(unvisited):
            heapq.heappop(unvisited)

        # rebuild unvisited
        unvisited = [(v.getDistance(), v) for v in graph if not v.getVisited()]

    return graph.vertDict



In [76]:
import heapq
import sys

def dijkstra(graph, s):
    """Computes the shortest distance from source vertex to all other vertices."""
    start = graph.getVertix(s)
    start.setDistance(0)
    
    # Priority queue (min-heap)
    unvisited = []
    heapq.heappush(unvisited, (start.getDistance(), start))  # Push the starting node

    while unvisited:
        curr_distance, curr = heapq.heappop(unvisited)  # Extract the closest vertex
        curr.setVisited()

        for neighbor in curr.adjacent:
            next_vertix = graph.getVertix(neighbor)

            if next_vertix.getVisited():
                continue

            new_distance = curr.getDistance() + curr.getWeight(neighbor)

            if new_distance < next_vertix.getDistance():
                next_vertix.setDistance(new_distance)
                next_vertix.setPrev(curr)
                heapq.heappush(unvisited, (new_distance, next_vertix))  # Push updated distance

    return graph.vertDict  # Returns the dictionary of all vertices with updated distances


In [77]:
# example
graph = Graph()

graph.addEdge('a', 'b', 4)
graph.addEdge('a', 'c', 1)
graph.addEdge('c', 'b', 2)
graph.addEdge('c', 'd', 4)
graph.addEdge('b', 'e', 4)
graph.addEdge('d', 'e', 4)

# print(graph.getEdges())

graph.vertDict

{'a': <__main__.Vertix at 0x1eabb852850>,
 'b': <__main__.Vertix at 0x1eabb853310>,
 'c': <__main__.Vertix at 0x1eabb853ad0>,
 'd': <__main__.Vertix at 0x1eabb851410>,
 'e': <__main__.Vertix at 0x1eabb852dd0>}

In [78]:
for i in graph.vertDict.values():
    print(i.getDistance())

9223372036854775807
9223372036854775807
9223372036854775807
9223372036854775807
9223372036854775807


In [79]:
unvisited = [(v.getDistance(), v) for v in graph if not v.getVisited()]
unvisited

[(9223372036854775807, <__main__.Vertix at 0x1eabb852850>),
 (9223372036854775807, <__main__.Vertix at 0x1eabb853310>),
 (9223372036854775807, <__main__.Vertix at 0x1eabb853ad0>),
 (9223372036854775807, <__main__.Vertix at 0x1eabb851410>),
 (9223372036854775807, <__main__.Vertix at 0x1eabb852dd0>)]

In [83]:
for vertix in dijkstra(graph, 'b').values():
    print(vertix.getVertixId(), vertix.getDistance())


a 9223372036854775807
b 0
c 9223372036854775807
d 9223372036854775807
e 4


In [88]:
import heapq
import sys

def dijkstra_(graph, s):
    start = graph.getVertix(s)
    start.setDistance(0)
    start.setPrev(None)
    unvisited = []
    heapq.heappush(unvisited, (start.getDistance(), start))

    while unvisited:
        curr_distance, curr = heapq.heappop(unvisited)
        curr.setVisited()

        for next in curr.adjacent:
            next_vertix = graph.getVertix(next)

            if next_vertix.getVisited():
                continue

            newDistance = curr.getDistance() + curr.getWeight(next)
            if newDistance < next_vertix.getDistance():
                next_vertix.setDistance(newDistance)
                next_vertix.setPrev(curr)
                heapq.heappush(unvisited, (next_vertix.getDistance(), next_vertix))

    return graph.vertDict

In [89]:
for vertix in dijkstra_(graph, 'b').values():
    print(vertix.getVertixId(), vertix.getDistance())

a 9223372036854775807
b 0
c 9223372036854775807
d 9223372036854775807
e 4


In [90]:
# for negative edge and negative cycle detection
class Graph(object):
    def __init__(self, vertices):
        self.V = vertices
        self.edges = []

    def addEdge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def bellmanford(self, src):
        # initialize distances
        distance = {v: sys.maxsize for v in range(self.V)}
        distance[src] = 0
        predessor = {v : None for v in range(self.V)}

        # relax the vertix V-1 times
        for _ in range(self.V - 1):
            for u, v, weight in self.edges:
                if distance[u] != sys.maxsize and distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight
                    predessor[v] = u

        # detect negative cycle
        for u, v, weight in self.edges:
            if distance[u] != sys.maxsize and distance[u] + weight < distance[v]:
                print("There is a negative cycle")
                return None
        
        return distance, predessor